# random - Pseudorandom Number Generators

Purpose:	Implements several types of pseudorandom number generators.

The random module provides a fast pseudorandom number generator based on the Mersenne Twister algorithm. Originally developed to produce inputs for Monte Carlo simulations, Mersenne Twister generates numbers with nearly uniform distribution and a large period, making it suited for a wide range of applications.

## Generating Random Numbers

The random() function returns the next random floating point value from the generated sequence. All of the return values fall within the range 0 <= n < 1.0.

Running the program repeatedly produces different sequences of numbers.

In [1]:
# random_random.py

import random

for i in range(5):
    print('%04.3f' % random.random(), end=' ')
print()

0.279 0.725 0.782 0.436 0.120 


To generate numbers in a specific numerical range, use uniform() instead.

Pass minimum and maximum values, and uniform() adjusts the return values from random() using the formula min + (max - min) * random().

In [2]:
# random_uniform.py

import random

for i in range(5):
    print('{:04.3f}'.format(random.uniform(1,100)), end=' ')

99.926 20.152 66.598 53.730 30.377 

## Seeding

random() produces different values each time it is called and has a very large period before it repeats any numbers. This is useful for producing unique values or variations, but there are times when having the same data set available to be processed in different ways is useful. One technique is to use a program to generate random values and save them to be processed by a separate step. That may not be practical for large amounts of data, though, so random includes the seed() function for initializing the pseudorandom generator so that it produces an expected set of values.

The seed value controls the first value produced by the formula used to produce pseudorandom numbers, and since the formula is deterministic it also sets the full sequence produced after the seed is changed. The argument to seed() can be any hashable object. The default is to use a platform-specific source of randomness, if one is available. Otherwise, the current time is used.

In [3]:
# random_seed.py

import random

random.seed(1)

for i in range(5):
    print('{:04.3f}'.format(random.random()), end = ' ')

0.134 0.847 0.764 0.255 0.495 

## Saving State

The internal state of the pseudorandom algorithm used by random() can be saved and used to control the numbers produced in subsequent runs. Restoring the previous state before continuing reduces the likelihood of repeating values or sequences of values from the earlier input. The getstate() function returns data that can be used to re-initialize the random number generator later with setstate().

The data returned by getstate() is an implementation detail, so this example saves the data to a file with pickle but otherwise treats it as a black box. If the file exists when the program starts, it loads the old state and continues. Each run produces a few numbers before and after saving the state, to show that restoring the state causes the generator to produce the same values again.

In [4]:
# random_state.py

import random
import os
import pickle

if os.path.exists('state.dat'):
    # Restore the previously saved state
    print('Found state.dat, initializing random module')
    with open('state.dat', 'rb') as f:
        state = pickle.load(f)
    random.setstate(state)
else:
    # Use a well-known start state
    print('No state.dat, seeding')
    random.seed(1)

# Produce random values
for i in range(3):
    print('{:04.3f}'.format(random.random()), end =' ')
print()

# save state for next time
with open('state.dat', 'wb') as f:
#     print(random.getstate())
    pickle.dump(random.getstate(), f)

# produce more random values
print('\n After saving state: ')
for i in range(3):
    print('{:04.3f}'.format(random.random()), end=' ')
print()

No state.dat, seeding
0.134 0.847 0.764 

 After saving state: 
0.255 0.495 0.449 


In [5]:
! python3 random_state.py

Found state.dat, initializing random module
0.255 0.495 0.449 

 After saving state: 
0.652 0.789 0.094 


In [6]:
! python3 random_state.py

Found state.dat, initializing random module
0.652 0.789 0.094 

 After saving state: 
0.028 0.836 0.433 


In [7]:
!rm state.dat

## Random Integers

random() generates floating point numbers. It is possible to convert the results to integers, but using randint() to generate integers directly is more convenient.

The arguments to randint() are the ends of the inclusive range for the values. The numbers can be positive or negative, but the first value should be less than the second.

In [8]:
# random_randint.py

import random

print('{:<10}:'.format('[1, 100]'), end=' ')

for i in range(5):
    print(random.randint(1, 100), end=' ')
    
print('\n{:<10}:'.format('[-5, 5 ]'), end=' ')

for i in range(5):
    print(random.randint(-5, 5), end=' ')

[1, 100]  : 84 49 27 13 63 
[-5, 5 ]  : -5 1 1 4 -5 

randrange() is a more general form of selecting values from a range.

randrange() supports a step argument, in addition to start and stop values, so it is fully equivalent to selecting a random value from range(start, stop, step). It is more efficient, because the range is not actually constructed.

In [9]:
# random_randrange.py

import random

for i in range(5):
    print(random.randrange(0, 101, 5), end=' ')

70 40 35 90 15 

## Picking Random Items

One common use for random number generators is to select a random item from a sequence of enumerated values, even if those values are not numbers. random includes the choice() function for making a random selection from a sequence. This example simulates flipping a coin 10,000 times to count how many times it comes up heads and how many times tails.

There are only two outcomes allowed, so rather than use numbers and convert them the words “heads” and “tails” are used with choice(). The results are tabulated in a dictionary using the outcome names as keys.

In [10]:
# random_choice.py

import random
import itertools

outcomes = {'head':0, 'tails':0}

sides = list(outcomes.keys())

for i in range(10000):
    outcomes[random.choice(sides)] += 1

for side in sides:
    print('{:<5}: {}'.format(side, outcomes[side]))

head : 5058
tails: 4942


## Permutations
A simulation of a card game needs to mix up the deck of cards and then deal them to the players, without using the same card more than once. Using choice() could result in the same card being dealt twice, so instead, the deck can be mixed up with shuffle() and then individual cards removed as they are dealt.

The cards are represented as strings with the face value and a letter indicating the suit. The dealt “hands” are created by adding one card at a time to each of four lists, and removing it from the deck so it cannot be dealt again.

In [11]:
# random_shuffle.py

import random
import itertools

FACE_CARDS = ('J', 'Q', 'K', 'A')
SUITES = ('H', 'D', 'C', 'S')

def new_deck():
    return[
        # Always use 2 places for the value, so the strings
        # are a consistent width.
        '{:>2}{}'.format(*c)
        for c in itertools.product(
            itertools.chain(range(2, 11), FACE_CARDS), SUITES,
        )
    ]


def show_deck(deck):
    p_deck = deck[:]
    while p_deck:
        row = p_deck[:10]
        p_deck = p_deck[10:]
        for j in row:
            print(j, end=' ')
        print()

# Make a new deck, with the cards in order
deck = new_deck()
print('Initial deck:')
show_deck(deck)

# Shuffle the deck to randomize the order
random.shuffle(deck)
print('\nShuffled deck:')
show_deck(deck)

# Deal 4 hands of 5 cards each
hands = [[], [], [], []]

for i in range(5):
    for h in hands:
        h.append(deck.pop())

# show the hands
print('\nHands:')
for n, h in enumerate(hands):
    print('{}:'.format(n + 1), end=' ')
    for c in h:
        print(c, end=' ')
    print()

# show the remaining deck
print('\nRemaining deck:')
show_deck(deck)

Initial deck:
 2H  2D  2C  2S  3H  3D  3C  3S  4H  4D 
 4C  4S  5H  5D  5C  5S  6H  6D  6C  6S 
 7H  7D  7C  7S  8H  8D  8C  8S  9H  9D 
 9C  9S 10H 10D 10C 10S  JH  JD  JC  JS 
 QH  QD  QC  QS  KH  KD  KC  KS  AH  AD 
 AC  AS 

Shuffled deck:
 JH  9D  2C  5H  8H  9C  KD  8D  7H  4C 
 4D  3S  4H  KC  3C  5D  AD  2H  QS  AH 
 JD  AS  QC  7C  JC  8C  7D  KH  7S  2S 
 4S  2D  5S  AC  8S  6D  9S  9H  6S 10H 
 6H  JS  3H 10D  3D  QD  6C  5C  KS 10S 
10C  QH 

Hands:
1:  QH  5C 10D 10H  6D 
2: 10C  6C  3H  6S  8S 
3: 10S  QD  JS  9H  AC 
4:  KS  3D  6H  9S  5S 

Remaining deck:
 JH  9D  2C  5H  8H  9C  KD  8D  7H  4C 
 4D  3S  4H  KC  3C  5D  AD  2H  QS  AH 
 JD  AS  QC  7C  JC  8C  7D  KH  7S  2S 
 4S  2D 


## Sampling

Many simulations need random samples from a population of input values. The sample() function generates samples without repeating values and without modifying the input sequence. This example prints a random sample of words from the system dictionary.

The algorithm for producing the result set takes into account the sizes of the input and the sample requested to produce the result as efficiently as possible.

In [12]:
# random_sample.py
import random

with open('/usr/share/dict/words', 'rt') as f:
    words = f.readlines()
words = [w.rstrip() for w in words]

for w in random.sample(words, 5):
    print(w)

smith
escapism
slummage
nonstandardized
vibrioid


## Multiple Simultaneous Generators

In addition to module-level functions, random includes a Random class to manage the internal state for several random number generators. All of the functions described earlier are available as methods of the Random instances, and each instance can be initialized and used separately, without interfering with the values returned by other instances.

On a system with good native random value seeding, the instances start out in unique states. However, if there is no good platform random value generator, the instances are likely to have been seeded with the current time, and therefore produce the same values.

In [13]:
# random_random_class.py

import random
import time

print('Default initialization:\n')

r1 = random.Random()
r2 = random.Random()

for i in range(3):
    print('{:04.3f}  {:04.3f}'.format(r1.random(), r2.random()))

print('\nSame seed:\n')

seed = time.time()
r1 = random.Random(seed)
r2 = random.Random(seed)

for i in range(3):
    print('{:04.3f}  {:04.3f}'.format(r1.random(), r2.random()))

Default initialization:

0.895  0.778
0.813  0.037
0.345  0.191

Same seed:

0.910  0.910
0.853  0.853
0.241  0.241


## SystemRandom

Some operating systems provide a random number generator that has access to more sources of entropy that can be introduced into the generator. random exposes this feature through the SystemRandom class, which has the same API as Random but uses os.urandom() to generate the values that form the basis of all of the other algorithms.

Sequences produced by SystemRandom are not reproducible because the randomness is coming from the system, rather than software state (in fact, seed() and setstate() have no effect at all).

In [14]:
# random_system_random.py
import random
import time

print('Default initializiation:\n')

r1 = random.SystemRandom()
r2 = random.SystemRandom()

for i in range(3):
    print('{:04.3f}  {:04.3f}'.format(r1.random(), r2.random()))

print('\nSame seed:\n')

seed = time.time()
r1 = random.SystemRandom(seed)
r2 = random.SystemRandom(seed)

for i in range(3):
    print('{:04.3f}  {:04.3f}'.format(r1.random(), r2.random()))

Default initializiation:

0.019  0.170
0.491  0.352
0.730  0.226

Same seed:

0.190  0.732
0.351  0.308
0.599  0.114


## Non-uniform Distributions

While the uniform distribution of the values produced by random() is useful for a lot of purposes, other distributions more accurately model specific situations. The random module includes functions to produce values in those distributions, too. They are listed here, but not covered in detail because their uses tend to be specialized and require more complex examples.

### Normal

The normal distribution is commonly used for non-uniform continuous values such as grades, heights, weights, etc. The curve produced by the distribution has a distinctive shape which has lead to it being nicknamed a “bell curve.” random includes two functions for generating values with a normal distribution, normalvariate() and the slightly faster gauss() (the normal distribution is also called the Gaussian distribution).

The related function, lognormvariate() produces pseudorandom values where the logarithm of the values is distributed normally. Log-normal distributions are useful for values that are the product of several random variables which do not interact.

### Approximation

The triangular distribution is used as an approximate distribution for small sample sizes. The “curve” of a triangular distribution has low points at known minimum and maximum values, and a high point at the mode, which is estimated based on a “most likely” outcome (reflected by the mode argument to triangular()).

### Exponential

expovariate() produces an exponential distribution useful for simulating arrival or interval time values for in homogeneous Poisson processes such as the rate of radioactive decay or requests coming into a web server.

The Pareto, or power law, distribution matches many observable phenomena and was popularized by The Long Tail, by Chris Anderson. The paretovariate() function is useful for simulating allocation of resources to individuals (wealth to people, demand for musicians, attention to blogs, etc.).

### Angular

The von Mises, or circular normal, distribution (produced by vonmisesvariate()) is used for computing probabilities of cyclic values such as angles, calendar days, and times.

### Sizes

betavariate() generates values with the Beta distribution, which is commonly used in Bayesian statistics and applications such as task duration modeling.

The Gamma distribution produced by gammavariate() is used for modeling the sizes of things such as waiting times, rainfall, and computational errors.

The Weibull distribution computed by weibullvariate() is used in failure analysis, industrial engineering, and weather forecasting. It describes the distribution of sizes of particles or other discrete objects.